In [12]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
print(os.listdir('/content/gdrive/My Drive/face_mask-dataset/train'))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['without_mask', 'with_mask']


In [2]:
import tensorflow as tf
from keras import Model, Sequential, layers, models
from keras.applications import MobileNetV2
from keras.optimizers import Adam
from PIL import ImageFile
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
image_width = 224 
image_height = 224 
batch_size = 32
epochs = 20
initial_lr = 1e-4
train_path = '/content/gdrive/My Drive/face_mask-dataset/train'
valid_path = '/content/gdrive/My Drive/face_mask-dataset/valid'

In [4]:
base_model = MobileNetV2(weights='imagenet', include_top=False,input_tensor= layers.Input(shape=(224,224,3)))
head_model = base_model.output
head_model = layers.AveragePooling2D((7,7))(head_model)
head_model = layers.Flatten(name='Flatten')(head_model)
head_model = layers.Dense(128,activation='relu')(head_model)
head_model = layers.Dropout(0.5)(head_model)
head_model = layers.Dense(2, activation='softmax')(head_model)

model = Model(inputs=base_model.input, outputs= head_model)

for layer in base_model.layers:
  layer.trainable=False

opt = Adam(learning_rate=initial_lr, decay = initial_lr/epochs)
model.compile(loss='binary_crossentropy',optimizer= opt, metrics =['accuracy'])

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 0s 0us/step


In [5]:
train_data_generator = ImageDataGenerator(
    horizontal_flip = True, 
    rescale = 1./255,
    rotation_range = 20,
    zoom_range=0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range=0.15,
    fill_mode = "nearest"
)

valid_data_generator = ImageDataGenerator(
    rescale = 1./255
)
train_generator = train_data_generator.flow_from_directory(
    train_path, 
    target_size = (image_width, image_width),
    class_mode = 'categorical'
)
valid_generator = valid_data_generator.flow_from_directory(
    valid_path,
    target_size = (image_width, image_width),
    class_mode = 'categorical'

)

Found 1101 images belonging to 2 classes.
Found 275 images belonging to 2 classes.


In [16]:
model.fit_generator(train_generator, 
                    steps_per_epoch=552//batch_size,
                    validation_data = valid_generator,
                    validation_steps = 138//batch_size,
                    epochs = 50
                    )

Epoch 1/50
17/17 [==============================] - 62s 4s/step - loss: 0.1235 - accuracy: 0.9596 - val_loss: 0.0606 - val_accuracy: 0.9609
Epoch 2/50
17/17 [==============================] - 54s 3s/step - loss: 0.1451 - accuracy: 0.9505 - val_loss: 0.0544 - val_accuracy: 0.9609
Epoch 3/50
17/17 [==============================] - 58s 3s/step - loss: 0.1371 - accuracy: 0.9448 - val_loss: 0.0895 - val_accuracy: 0.9652
Epoch 4/50
17/17 [==============================] - 61s 4s/step - loss: 0.1298 - accuracy: 0.9485 - val_loss: 0.0423 - val_accuracy: 0.9844
Epoch 5/50
17/17 [==============================] - 66s 4s/step - loss: 0.1176 - accuracy: 0.9651 - val_loss: 0.0331 - val_accuracy: 0.9826
Epoch 6/50
17/17 [==============================] - 43s 3s/step - loss: 0.1303 - accuracy: 0.9524 - val_loss: 0.0321 - val_accuracy: 0.9922
Epoch 7/50
17/17 [==============================] - 49s 3s/step - loss: 0.1145 - accuracy: 0.9504 - val_loss: 0.0112 - val_accuracy: 0.9652
Epoch 8/50
17/17 [==

In [17]:
model.save('face_mask_detection.h5')
model.save_weights('face_mask_detection_weights.h5')
model_json = model.to_json()
with open("face_mask_detection.json", "w") as json_file:
    json_file.write(model_json)

model.save('mask_detector.model')

In [15]:
model.save('mask_detector.model')